In [1]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Input, BatchNormalization, Dropout, Dense

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
 tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [23]:
train_path = 'train'
validation_path = 'val'
test_path = 'test'

def datagen_function(path, aug=False):
    if aug == True:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0,
            rotation_range = 10,
            zoom_range = 0.1,
            horizontal_flip = True
        )
    else:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0
        )
    return generator.flow_from_directory(
        path,
        target_size = (48,48),
        color_mode = 'rgb',
        shuffle = True,
        class_mode = 'categorical',
        batch_size = 64
    )

In [24]:
train_gen = datagen_function(train_path, True)
validation_gen = datagen_function(validation_path)
test_gen = datagen_function(test_path)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [29]:
# vgg-16 model definition

input = Input(shape = (48, 48, 3))

base_vgg16 = tf.keras.applications.VGG16(include_top = False, weights = 'imagenet', input_shape = (224, 224, 3), pooling = 'max',classes = 7)
for layer in base_vgg16.layers:
    layer.trainable = False

x = base_vgg16(input)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)

x = Dense(7,activation='softmax')(x)
model = Model(input, x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_7 (Flatten)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 4096)              2101248   
                                                                 
 dropout_3 (Dropout)         (None, 4096)              0         
                                                                 
 dense_16 (Dense)            (None, 1024)              4195328   
                                                                 
 dense_17 (Dense)            (None, 7)                 7175

In [30]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 5, min_lr = 0.000001, verbose = 1)
checkpoint = ModelCheckpoint('ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5', monitor='val_acc', save_best_only=True, save_freq="epoch")

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(loss = tf.keras.losses.categorical_crossentropy, metrics = 'acc', optimizer = optimizer)
history = model.fit(train_gen,validation_data = validation_gen, epochs = 100, callbacks = [reduce_lr, checkpoint])

Epoch 1/100
449/449 [==============================] - 313s 683ms/step - loss: 1.7327 - acc: 0.3142 - val_loss: 1.6415 - val_acc: 0.3514 - lr: 0.0010
Epoch 2/100
449/449 [==============================] - 305s 678ms/step - loss: 1.6063 - acc: 0.3650 - val_loss: 1.5968 - val_acc: 0.3851 - lr: 0.0010
Epoch 3/100
449/449 [==============================] - 298s 663ms/step - loss: 1.5780 - acc: 0.3848 - val_loss: 1.5403 - val_acc: 0.3987 - lr: 0.0010
Epoch 4/100
449/449 [==============================] - 298s 664ms/step - loss: 1.5557 - acc: 0.3966 - val_loss: 1.5417 - val_acc: 0.3937 - lr: 0.0010
Epoch 5/100
449/449 [==============================] - 297s 660ms/step - loss: 1.5567 - acc: 0.3927 - val_loss: 1.5736 - val_acc: 0.3803 - lr: 0.0010
Epoch 6/100
449/449 [==============================] - 297s 661ms/step - loss: 1.5400 - acc: 0.3990 - val_loss: 1.5337 - val_acc: 0.4051 - lr: 0.0010
Epoch 7/100
449/449 [==============================] - 298s 662ms/step - loss: 1.5308 - acc: 0.4029 

KeyboardInterrupt: 